#Create Corpus

[CSV file with Azure & Microsoft links created](https://docs.google.com/spreadsheets/d/1b_QcHNPBg34Q05FPsmRqzPW5XTUGmedaP_SLonyiMt4/edit?usp=sharing)

Columns:
- Name
- URL
- Content type

In [ ]:
# Install requirements

!pip install -q llama-index==0.12.12 openai==1.59.6 tiktoken==0.8.0 llama-index-readers-web==0.3.4 firecrawl-py==2.7.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 kB 3.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 77.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 454.8/454.8 kB 27.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 45.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.2/80.2 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.2/71.2 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 77.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 266.8/266.8 kB 17.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.0/41.0 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2

In [ ]:
# set variables
import os

from google.colab import userdata

os.environ["OPENAI_API_KEY"] = userdata.get('openai_api_key')
os.environ["FIRECRAWL_API_KEY"] = userdata.get('FIRECRAWL_API_KEY')
os.environ["HF_TOKEN"] = userdata.get('HF_TOKEN2')

FIRECRAWL_API_KEY = userdata.get('FIRECRAWL_API_KEY')
HF_TOKEN = userdata.get('HF_TOKEN2')

In [ ]:
# Download dataset list

import requests
import csv

# Google Sheets file URL (CSV export link)
url = 'https://docs.google.com/spreadsheets/d/1b_QcHNPBg34Q05FPsmRqzPW5XTUGmedaP_SLonyiMt4/export?format=csv'

# Send a GET request to fetch the CSV file
response = requests.get(url)

response_list = []

# Check if the request was successful
if response.status_code == 200:
    # Decode the content to a string
    content = response.content.decode('utf-8')

    # Use the csv.DictReader to read the content as a dictionary
    csv_reader = csv.DictReader(content.splitlines(), delimiter=',')
    response_list = [row for row in csv_reader]
else:
    print(f"Failed to retrieve the file: {response.status_code}")


In [ ]:
import pprint
print("CSV data")
pprint.pprint(response_list[2:3])

CSV data
[{'Content type': 'Technical blogs and expert solutions',
  'Exclude': '',
  'Name': 'Tech Community',
  'URL': 'https://techcommunity.microsoft.com/'}]


In [ ]:
# Initialise Firecrawl
import os
from firecrawl import FirecrawlApp

app = FirecrawlApp(api_key=FIRECRAWL_API_KEY)



In [ ]:
import time
from firecrawl import ScrapeOptions

# Crawl websites and handle responses
url_response = {}
crawl_per_min = 15  # Max crawl per minute

# Track crawls
crawled_websites = 0
scraped_pages = 0

for i, website_dict in enumerate(response_list[2:3]):
    url = website_dict.get('URL')
    excludePaths = website_dict.get('Exclude')
    print(f"Crawling: {url}")

    try:
        response = app.crawl_url(
            url,
            limit=15000,
            scrape_options=ScrapeOptions(formats=['markdown']),
            exclude_paths=[excludePaths]
        )

        #'includePaths': [r"^https:\/\/[^\/]+(?:\/en-US(?:\/.*)?|(\/(?![a-z]{2}-[A-Z]{2})([^\/]+)?(?:\/[^\/]*)*)?)$"],
        crawled_websites += 1


    except Exception as exc:
        print(f"Failed to fetch {url} -> {exc}")
        continue

    # Store the scraped data and associated info in the response dict
    url_response[url] = {
        "scraped_data": response.data,
        "csv_data": website_dict
    }

    # Pause to comply with crawl per minute limit for free version its 1 crawl per minute
    if i!=len(response_list) and (i + 1) % crawl_per_min == 0:
        print("Pausing for 1 minute to comply with crawl limit...")
        time.sleep(60)  # Pause for 1 minute after every crawl

NameError: name 'response_list' is not defined

In [ ]:
# Initialise HG

from huggingface_hub import HfApi

api = HfApi(token=HF_TOKEN)



In [ ]:
from llama_index.core import Document
documents = []

for _, scraped_content in url_response.items():
    csv_data = scraped_content.get("csv_data")
    scraped_results = scraped_content.get("scraped_data")


    for result in scraped_results:
        markdown_content = result.markdown
        title = result.metadata.get("title")
        url = result.metadata.get("sourceURL")
        documents.append(
            Document(
                text=markdown_content,
                metadata={
                    "title": title,
                    "url": url,
                    "name": csv_data.get("Name"),
                    "contentType": csv_data.get("Content type")
                }
            )
        )

In [ ]:
print("Documents")
print( len(documents))
pprint.pprint(documents[0:10])

Documents
1
[Document(id_='59766750-ed2d-4359-98d0-a8024f5b7c3b', embedding=None, metadata={'title': 'Azure updates | Microsoft Azure', 'url': 'https://azure.microsoft.com/en-us/updates/', 'name': 'Azure Blog / Updates', 'contentType': 'Platform changes, new features'}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, metadata_template='{key}: {value}', metadata_separator='\n', text_resource=MediaResource(embeddings=None, data=None, text='Trace Id is missing\n\n[Skip to main content](javascript:void(0))\n\n![](https://cdn-dynmedia-1.microsoft.com/is/image/microsoftcorp/ACOM-Roadmap-hero?resMode=sharp2&op_usm=1.5,0.65,15,0&wid=1600&hei=635&qlt=85&fit=constrain)\n\n# Azure Updates\n\nGet the latest updates on Azure products and features to meet your cloud investment needs. Subscribe to notifications to stay informed.\n\n[Get the user guide](https://go.microsoft.com/fwlink/?linkid=2289891&clcid=0x409)[Subscribe via RSS](https://www.microsoft.com/releasecom

In [ ]:
# prompt: Write documents array to csv file

# Define the CSV filename for the documents
filename = 'dataset/documents.csv'
directory_name = "dataset"

# Comprueba si el directorio no existe
if not os.path.exists(directory_name):
  # Crea el directorio
  os.makedirs(directory_name)
  print(f"Directorio '{directory_name}' creado.")
else:
  print(f"El directorio '{directory_name}' ya existe.")

# Ensure documents is a list before proceeding
if isinstance(documents, list):
  # Open the file in write mode ('w')
  # newline='' avoids issues with blank lines
  with open(filename, 'w', newline='') as file:
    writer = csv.writer(file)
    # Write the header row
    writer.writerow(['Text', 'Title', 'URL', 'Name', 'ContentType'])
    # Write each document as a row
    for doc in documents:
      writer.writerow([
          doc.text,
          doc.metadata.get('title'),
          doc.metadata.get('url'),
          doc.metadata.get('name'),
          doc.metadata.get('contentType')
      ])

  print(f"The documents array has been saved to '{filename}'")
else:
  print("documents is not a list. Cannot write to CSV.")

# Upload the new documents CSV to Hugging Face
api.upload_file(
    path_or_fileobj=filename,
    path_in_repo='documents.csv',
    repo_id="vicpada/AzureResources",
    repo_type="dataset",
)

Directorio 'dataset' creado.
The documents array has been saved to 'dataset/documents.csv'


documents.csv:   0%|          | 0.00/44.7k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/datasets/vicpada/AzureResources/commit/6e04a0cabff20f42d9f440972f7fec91caa5cb45', commit_message='Upload documents.csv with huggingface_hub', commit_description='', oid='6e04a0cabff20f42d9f440972f7fec91caa5cb45', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/vicpada/AzureResources', endpoint='https://huggingface.co', repo_type='dataset', repo_id='vicpada/AzureResources'), pr_revision=None, pr_num=None)

In [ ]:
# prompt: write a dictionary to a csv file

import csv

directory_name = "dataset"

# Comprueba si el directorio no existe
if not os.path.exists(directory_name):
  # Crea el directorio
  os.makedirs(directory_name)
  print(f"Directorio '{directory_name}' creado.")
else:
  print(f"El directorio '{directory_name}' ya existe.")

# save to CSV
filename = 'dataset/website-content.csv'

# Ensure url_response is a dictionary before proceeding
if isinstance(url_response, dict):
  # Open the file in write mode ('w')
  # newline='' avoids issues with blank lines
  with open(filename, 'w', newline='') as file:
    writer = csv.writer(file)
    # Write the header row
    writer.writerow(['URL', 'Scraped Data', 'CSV Data'])
    # Write each row from the dictionary
    for url, data in url_response.items():
      writer.writerow([url, data.get('scraped_data'), data.get('csv_data')])

  print(f"El array ha sido guardado en '{filename}'")
else:
  print("url_response is not a dictionary. Cannot write to CSV.")

  # Upload to HG

api.upload_folder(
    folder_path="dataset",
    repo_id="vicpada/AzureResources",
    repo_type="dataset",
)

El directorio 'dataset' ya existe.
El array ha sido guardado en 'dataset/website-content.csv'
